In [ ]:
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt

In [ ]:
# загрузка предобученных моделей
models_dir = os.listdir('PATH_to_FOLDER')
members = []
for i in range(len(models_dir)):
  model = torch.jit.load('PATH_to_FOLDER/' + models_dir[i])
  model.eval()
  members.append(model)
n_members = len(members)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# загрузка данных
data_test = np.load('PATH_to_DATA_test')
masks_test = np.load('PATH_to_MASKS_test')

data = DataLoader(list(zip(data_test, masks_test)),
                     batch_size=16, shuffle=False,
                     pin_memory=True)

# функция вычисления метрики IoU по результатам одной модели
def check_accuracy(loader, model):
    iou_score = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            softmax = nn.Softmax(dim=1)
            preds = torch.argmax(softmax(model(x)),axis=1)
            iou_score += (((preds & y).float().sum((1, 2)) + 1e-8) / ((preds | y).float().sum((1, 2)) + 1e-8)).mean().item()
    model.train()
    return iou_score/len(loader)

# функция вычисления выхода модели для одного изображения
def predictions(model, x):
    softmax = nn.Softmax(dim=1)
    preds = softmax(model(x)).to('cpu').detach().numpy()
    preds = np.array(preds)
    preds = np.rollaxis(preds, 1, 4)
    return preds

# функция расчета выхода ансамбля с весами моделей
def ensemble_predictions(preds, weights):
    summed = np.tensordot(preds, weights, axes=((0,0)))
    result = np.argmax(summed, axis = 3)
    return result

# функция вычисления выходов модели для всего датасета
def general_predictions(model, data):
  preds = []
  for x, y in data:
    x = x.to(device)
    preds.append(predictions(model, x))
  preds = np.reshape(np.array(preds), (len(data)*16, 256, 256, 2))
  return preds

# функция вычисления метрики IoU для результатов ансамбля
def evaluate_ensemble(preds, targets, weights):
  iou_score = 0
  preds = ensemble_predictions(preds, weights)
  iou_score = (((preds & targets).sum((1, 2)) + 1e-8) / ((preds | targets).sum((1, 2)) + 1e-8)).mean()
  return iou_score

# функция нормализации весов
def normalize(weights):
  result = np.linalg.norm(weights, 1)
  if result == 0.0:
    return weights
  return weights / result

# минимизируемая функция
def loss_function(weights, preds, targets):
  normalized = normalize(weights)
  return 1.0 - evaluate_ensemble(preds, targets, normalized)

In [ ]:
# вычисление результатов отдельных моделей
preds = [general_predictions(model, data) for model in members]
targets = []
for x, y in data:
  targets.append(np.array(y))
targets = np.reshape(np.array(targets), (len(data)*16, 256, 256))
preds, targets = np.array(preds), np.array(targets)

# расчет метрики IoU для отдельных моделей
for i in range(n_members):
  score = check_accuracy(data, members[i])
  print('Model %d: %.3f' % (i+1, score))

# расчет метрики IoU для усредненного ансамбля
weights = [1.0/n_members for _ in range(n_members)]
score = evaluate_ensemble(preds, targets, weights)
print('Equal Weights Score: %.3f' % score)

# расчет оптимальных весов для средневзвешенного ансамбля и расчет метрики IoU
bound_w = [(0.0, 1.0) for _ in range(n_members)]
search_arg = (preds, targets)
result = differential_evolution(loss_function, bound_w, search_arg, maxiter = 1000, tol=1e-7)
weights_opt = normalize(result['x'])
print('Optimized Weights: %s' % weights)
score = evaluate_ensemble(members, weights, data)
print('Optimized Weight Score: %.3f' % score)

In [ ]:
# визуализация результатов отдельных моделей и ансамблей на тестовых изображениях
def rap(model, x):
  preds = softmax(model(x)).to('cpu').detach().numpy()
  preds = np.array(preds)
  preds = np.rollaxis(preds, 1, 4)
  return preds

for x, y in data:
  x = x.to(device)
  fig , ax =  plt.subplots(3, 3, figsize=(8, 8))
  softmax = nn.Softmax(dim=1)
  predso = [torch.argmax(softmax(model(x)),axis=1).to('cpu') for model in members]
  weights = [1.0/n_members for _ in range(n_members)]
  preds = [rap(model, x) for model in members]
  summed = np.tensordot(preds, weights, axes=((0,0)))
  result = np.argmax(summed, axis = 3)
  summed_opt = np.tensordot(preds, weights_opt, axes=((0,0)))
  result_opt = np.argmax(summed_opt, axis = 3)
  mask = np.array(y[1,:,:])
  preds1 = np.array(predso[0][1,:,:])
  preds2 = np.array(predso[1][1,:,:])
  preds3 = np.array(predso[2][1,:,:])
  preds4 = np.array(predso[3][1,:,:])
  preds5 = np.array(predso[4][1,:,:])
  preds6 = np.array(predso[5][1,:,:])
  preds_ensemble = np.array(result[1,:,:])
  preds_w_ensemble = np.array(result_opt[1,:,:])

  ax[0,0].set_title('Mask')
  ax[0,1].set_title('Weighted Average Ensemble')
  ax[0,2].set_title('Pred_SegNet')
  ax[1,0].set_title('Pred_UNet')
  ax[1,1].set_title('Pred_DDRNet')
  ax[1,2].set_title('Average Ensemble')
  ax[2,0].set_title('Pred_resnet')
  ax[2,1].set_title('Pres_FCN8s')
  ax[2,2].set_title('Pred_DeepLabV3+')
  ax[0][0].axis("off")
  ax[1][0].axis("off")
  ax[0][1].axis("off")
  ax[1][1].axis("off")
  ax[0][2].axis("off")
  ax[1][2].axis("off")
  ax[2][0].axis("off")
  ax[2][1].axis("off")
  ax[2][2].axis("off")
  ax[0][0].imshow(mask)
  ax[0][1].imshow(preds_w_ensemble)
  ax[0][2].imshow(preds2)
  ax[1][0].imshow(preds1)
  ax[1][1].imshow(preds4)
  ax[1][2].imshow(preds_ensemble)
  ax[2][0].imshow(preds6)
  ax[2][1].imshow(preds5)
  ax[2][2].imshow(preds3)